In [2]:
from cdlib import algorithms
import networkx.algorithms.community as nx_comm
from cdlib import viz
import networkx as nx
from cdlib import NodeClustering
import random
import numpy as np
import pandas as pd
from itertools import product
import time
import sys
from collections import defaultdict
from networkx.utils.mapped_queue import MappedQueue
import copy
from random import choices

In [3]:
def cetak(s, communities):
    comm= []
    for x in communities:
        if list(x)!=[]:
            comm.append(list(x))
    print (s,' : ',comm)
    #hasil = sorted((c for c in communities if len(c) > 0), key=len, reverse=True)
    #print(hasil)
    
        
def cetak_dq_dict(dq_dict): 
    print("\ndq_dict")
    for u, nbrdict in dq_dict.items():        
        lx = [x for x in dq_dict[u].items()]
        print(u, lx )    
        
def reformat2(G,coms): 
    hasil=[]
    for key in coms: #a_dict[key])
        if len(coms[key])>0:
            hasil.append(list(coms[key]))
    return(hasil)

def cetak_mapped_queue(HH):
    print("\nMapped Queue: ",[HH.pop().element for i in range(len(HH.heap))])
    
def cetak_dq_heap(cdq_heap):
    print("\ndq_heap,dengan panjang elemen",len(cdq_heap))
    for ii in range(len(cdq_heap)):    
        try:
            print(ii,[cdq_heap[ii].pop().element for i in range(len(cdq_heap[ii].heap))])
        except:
            print(ii)    

def createcom(listcom):
    dictcom={}
    for i in range(len(listcom)):
        dictcom[max(listcom[i])]=frozenset(listcom[i])
    return dictcom    

def hitungmodularitas(G,communities):
    listcom=reformat2(G,communities)
    modularitas = nx.community.modularity(G,listcom)
    return modularitas
    

def updatekomunitasterbaik(G,communities, komunitasterbaik, modularitasterbaik):
    modularitas = hitungmodularitas(G,communities)
    if modularitasterbaik < modularitas:
        komunitasterbaik=copy.deepcopy(communities)
        modularitasterbaik=modularitas
        #print("update modularitas terbaik", modularitasterbaik)
        #print("update komunitas terbaik", komunitasterbaik)
    return(komunitasterbaik, modularitasterbaik)

In [4]:
def pecah_random(communities):
    tempcom={}
    for x in communities:
        if len(communities[x])>1:
            tempcom[x]=communities[x]
    if len(tempcom)>=1:
        #print(dictcom)
        kunci, nilai = random.choice(list(tempcom.items()))
        communities.pop(kunci)   # menghapus elemen komunitas
        listnilai = [*nilai, ]
        for el in listnilai:
            communities[el] = frozenset({el})
    return (communities)

def pecah_weak(communities):
    dictcom={}
    for x in communities:
        if len(communities[x])>1:
            dictcom[x]=communities[x]
    dict_weak={} 
    #print(len(dictcom))
    if (len(dictcom)>=1):
        for c in dictcom:
            SG = G.subgraph(communities[c]) 
            #ms = len(SG.edges())
            edges_outside = 0
            for n in SG.nodes():
                neighbors = G.neighbors(n)
                for n1 in neighbors:
                    if n1 not in SG:
                        edges_outside += 1
            edges_inside=SG.number_of_edges()
            delta = edges_inside - edges_outside        
            dict_weak[c]=delta    
        kunci = max(dict_weak,key=dict_weak.get)
        nilai=dictcom[kunci]
        communities.pop(kunci)   # menghapus elemen komunitas
        c_star = [*nilai, ]
        for el in c_star:
            communities[el] = frozenset({el})
    return (communities)

def pecah_high_conductance(communities):
    dictcom={}
    for x in communities:
        if len(communities[x])>1:
            dictcom[x]=communities[x]
    dict_conductance={} 
    if (len(dictcom)>=1):
        for c in dictcom:
            SG = G.subgraph(communities[c]) 
            ms = len(SG.edges())
            edges_outside = 0
            for n in SG.nodes():
                neighbors = G.neighbors(n)
                for n1 in neighbors:
                    if n1 not in SG:
                        edges_outside += 1  
            try:
                scoreconductance = float(edges_outside) /((2 * ms) + edges_outside)
            except:
                scoreconductance = 0  
            dict_conductance[c]=scoreconductance
        #print("dict conductance",dict_conductance)
        kunci = max(dict_conductance,key=dict_conductance.get)
        nilai=dictcom[kunci]
        communities.pop(kunci)   # menghapus elemen komunitas
        c_star = [*nilai, ]
        for el in c_star:
            communities[el] = frozenset({el})                
    return (communities)

def pecah_low_tpr(communities):
    dictcom={}
    for x in communities:
        if len(communities[x])>1:
            dictcom[x]=communities[x]
    dict_tpr={} 
    if (len(dictcom)>=1):
        for c in dictcom:
            SG = G.subgraph(communities[c])             
            dict_triangle = nx.triangles(SG)
            node_triangle = [n for n in dict_triangle if dict_triangle[n] > 0]
            scoretpr = float(len(node_triangle)) / len(SG)           
            dict_tpr[c]=scoretpr 
        #print("dict tpr", dict_tpr)
        kunci = min(dict_tpr,key=dict_tpr.get)
        nilai=dictcom[kunci]
        communities.pop(kunci)   # menghapus elemen komunitas
        c_star = [*nilai, ]
        for el in c_star:
            communities[el] = frozenset({el})             
    return (communities)

def pecah_low_edge_density(communities):
    dictcom={}
    for x in communities:
        if len(communities[x])>1:
            dictcom[x]=communities[x]
    dict_density={} 
    if (len(dictcom)>=1):
        for c in dictcom:
            SG = G.subgraph(communities[c])
            density = nx.density(SG)   
            dict_density[c]=density
        #print("dict_density", dict_density)
        kunci = min(dict_density,key=dict_density.get)
        nilai=dictcom[kunci]
        communities.pop(kunci)   # menghapus elemen komunitas
        c_star = [*nilai, ]
        for el in c_star:
            communities[el] = frozenset({el})                         
    return (communities)

In [5]:
def lepas_random(communities):
    dictcom={}
    for x in communities:
        if len(communities[x])>1:
            dictcom[x]=communities[x]
    if len(dictcom)>=1:
        #print("alternatif  ",dictcom)
        kunci, nilai = random.choice(list(dictcom.items()))
        listnilai=[x for x in nilai ]
        el_lepas = random.choice(listnilai)
        #print("Yang dihapus ", kunci, listnilai, el_lepas )        
        listnilai.remove(el_lepas)
        communities.pop(kunci)
        communities[el_lepas] = frozenset({el_lepas})
        subgraphsisa =  G.subgraph(listnilai)
        listconns = [list(c) for c in nx.connected_components(subgraphsisa)]
        #print('list connected component',listconns)
        for  listconn in listconns:
            communities[max(listconn)]=frozenset(listconn)
        #print("hasil akhir:", communities)
    return (communities)

def lepas_weak_node(communities):
    dictcom={}
    for x in communities:
        if len(communities[x])>1:
            dictcom[x]=communities[x]
    if len(dictcom)>=1:
        for c in dictcom: 
            list_node_komunitas = [x for x in  communities[c]]
            #print("listnodekomunitas: ", list_node_komunitas)
            for node_komunitas in list_node_komunitas:
                list_edge_node_komunitas=list(G.edges(node_komunitas))
                in_comm = 0
                out_comm= 0
                for e in list_edge_node_komunitas:
                    if e[1] in list_node_komunitas:
                        in_comm += 1
                    else:
                        out_comm += 1
                #print(c,node_komunitas,in_comm, out_comm)
                if in_comm>=2 and out_comm>in_comm:
                    elemen=list(communities[c])
                    communities[c]=frozenset()
                    #print("elemen komunitas yang akan dilepas", elemen)
                    for el in elemen:
                        communities[el]=frozenset()
                    sisa=[item for item in elemen if item != node_komunitas]
                    communities[node_komunitas]=frozenset({node_komunitas})
                    subgraphsisa =  G.subgraph(sisa)
                    listconns = [list(c) for c in nx.connected_components(subgraphsisa)]
                    for  listconn in listconns:
                        communities[max(listconn)]=frozenset(listconn)
                    break;                 
    return (communities)

def lepas_non_triad_node(communities):
    dictcom={}
    for x in communities:
        if len(communities[x])>1:
            dictcom[x]=communities[x]
    if len(dictcom)>=1:
        for c in dictcom: 
            if len(communities[c])>=4:
                SG=G.subgraph(communities[c])
                SGTriangle = nx.triangles(SG)
                not_tpn = [n for n in SGTriangle if SGTriangle[n] == 0] 
                if len(not_tpn)>0:
                    listkom=list(communities[c])
                    communities[c]=frozenset()
                    for el in not_tpn:
                        communities[el]=frozenset({el})
                    sisa=[item for item in listkom if item not in not_tpn]
                    subgraphsisa =  G.subgraph(sisa)
                    listconns = [list(c) for c in nx.connected_components(subgraphsisa)]
                    for  listconn in listconns:
                        communities[max(listconn)]=frozenset(listconn)
                    break;
    return (communities)

def lepas_low_embeddedness_node(communities):
    global G
    #print("komunitas: ",communities)
    dictcom={}
    for x in communities:
        if len(communities[x])>1:
            dictcom[x]=communities[x]
    if len(dictcom)>=1:
        dictemb={}
        for c in dictcom:     
            listkom=list(communities[c])
            SG=G.subgraph(communities[c])
            for node in listkom:
                dictemb[node]=SG.degree[node]/G.degree[node]
        #print(dictemb)
        if len(dictemb)>0:
            dilepas = min(dictemb,key=dictemb.get) 
            #print("yang dilepas",dilepas)
            for key, value in communities.items():
                if dilepas in value:
                    nomor_komunitas=key
            #print("nomor komunitas",nomor_komunitas)             
 
            listkom=list(communities[nomor_komunitas])
            communities[nomor_komunitas]=frozenset({}) 
            communities[dilepas]=frozenset({dilepas})    

            #print("listkom ", listkom)
            sisa=[item for item in listkom if item != dilepas]
            #print("sisa ", sisa)
            subgraphsisa =  G.subgraph(sisa)
            listconns = [list(c) for c in nx.connected_components(subgraphsisa)]
            for  listconn in listconns:
                communities[max(listconn)]=frozenset(listconn)                     
    #print(communities)    
    return (communities)

In [6]:
def collapse_nodes_each(G,k,v):
    #print(k,v)
    edges = list(G.edges)
    new_edges = []
    for n1, n2 in edges:        
        if n1 in v:
            n1 = k
        if n2 in v:
            n2 = k
        if n1 == n2:
            continue
        new_edges.append((n1,n2))
        #print("n1 n2",n1,n2)
    H = nx.Graph()
    H.add_edges_from(new_edges)
    return H

def collapse_nodes(G, initcom):
    for  k,v in initcom.items():
        if len(v)>1:
            #print(k,list(v))        
            G=collapse_nodes_each(G,k,list(v))
    return G

def intersection(lst1, lst2):
    #https://stackoverflow.com/questions/54610818/difference-between-two-list-of-unordered-tuples
    if lst1 and lst2:
        #lst3 = [value for value in lst1 if value in lst2]
        #setlst3 = set.intersection(set(lst1), set(lst2))
        #lst3=list(setlst3)
        #lst3= np.intersect1d(lst1, lst2)
        
        lst3= set([tuple(sorted(elem))for elem in lst1]) & set([tuple(sorted(elem))for elem in lst2])
        #print(lst1, lst2, lst3)
    else:
        lst3=[]
    #print(lst1, lst2,lst3)
    return lst3


def alternative_intersection(lst1, lst2):
    if not lst1 or not lst2:
        return []

    # Mengurutkan dan menyimpan elemen unik dari lst1 dalam dictionary
    dict1 = {tuple(sorted(elem)): True for elem in lst1}

    # Mencari irisan dengan lst2
    intersection_set = set()
    for elem in lst2:
        sorted_elem = tuple(sorted(elem))
        if dict1.get(sorted_elem, False):
            intersection_set.add(sorted_elem)

    return list(intersection_set)


In [9]:
# yang iterasi greedy diambilkan dari greedy modularity yang telah ada sebelumnya
# https://networkx.org/documentation/stable/reference/algorithms/generated/
# networkx.algorithms.community.modularity_max.greedy_modularity_communities.html
def greedy_modularity_iteration(G, communities,weight=None, resolution=1):
    global GG, H, dq_heap, dq_dict, numedges, berhenti, mulai
    

    if mulai:
        GG = collapse_nodes(G, communities)
        m = G.size(weight)
        q0 = 1 / m
        a = b = {node: deg * q0 for node, deg in G.degree(weight=weight)}
        #print("nilai a",a)
        numedges=len(G.edges())
        dq_dict = defaultdict(lambda: defaultdict(float))
        for u, v, wt in GG.edges(data=weight, default=1):
            if u == v:
                continue
            elci = G.edges(list(communities[u]))
            elcj = G.edges(list(communities[v]))
            join=intersection(elci,elcj)
            if len(join)==0:
                continue   
            eij=len(join)/numedges    
            ai = len(G.edges(communities[u]))/numedges
            aj = len(G.edges(communities[v]))/numedges
            dq= 2*(eij-ai*aj) 
            dq_dict[u][v] = dq
            dq_dict[v][u] = dq
            #print("u, v, dq",u,v, dq)
        #cetak_dq_dict(dq_dict)

        dq_heap = {u: MappedQueue({(u, v): -dq for v, dq in dq_dict[u].items()}) for u in G}
        #cdq_heap = copy.deepcopy(dq_heap)
        #cetak_dq_heap(cdq_heap)

        H = MappedQueue([dq_heap[n].heap[0] for n in G if len(dq_heap[n]) > 0])
        berhenti=False

        #HH =copy.deepcopy(H)  
        #cetak_mapped_queue(HH)
        #print("mulai")
    else:
        berhenti=True
        if len(H) > 1:
            try:
                negdq, u, v = H.pop()
            except IndexError:
                return(communities, True)
            dq = -negdq
            #print("u v dq",u, v, dq)
            if dq>0:
                berhenti=False
                # Remove best merge from row u heap
                dq_heap[u].pop()
                # Push new row max onto H
                if len(dq_heap[u]) > 0:
                    H.push(dq_heap[u].heap[0])
                # If this element was also at the root of row v, we need to remove the
                # duplicate entry from H
                if dq_heap[v].heap[0] == (v, u):
                    H.remove((v, u))
                    # Remove best merge from row v heap
                    dq_heap[v].remove((v, u))
                    # Push new row max onto H
                    if len(dq_heap[v]) > 0:
                        H.push(dq_heap[v].heap[0])
                else:
                    # Duplicate wasn't in H, just remove from row v heap
                    dq_heap[v].remove((v, u))
                #cdq_heap = copy.deepcopy(dq_heap)
                #cetak_dq_heap(cdq_heap)       

                #print("u,v: ", u,v)
                #print("communities u, v SEBELUM : ", communities[u] , communities[v])            
                communities[v] = frozenset(communities[u] | communities[v])        
                del communities[u]

                #print("yang digabung ", max(list(communities[v])), list(communities[v]))            
                #print("communities", communities)
                GG=collapse_nodes_each(GG,max(list(communities[v])),list(communities[v]))

                # Get neighbor communities connected to the merged communities
                u_nbrs = set(dq_dict[u])
                v_nbrs = set(dq_dict[v])
                all_nbrs = (u_nbrs | v_nbrs) - {u, v}
                both_nbrs = u_nbrs & v_nbrs
                # Update dq for merge of u into v
                for w in all_nbrs:
                    # Calculate new dq value
                    #if w in both_nbrs:
                    #    dq_vw = dq_dict[v][w] + dq_dict[u][w]
                    #elif w in v_nbrs:
                    #    dq_vw = dq_dict[v][w] - resolution * (a[u] * b[w] + a[w] * b[u])
                    #else:  # w in u_nbrs
                    #    dq_vw = dq_dict[u][w] - resolution * (a[v] * b[w] + a[w] * b[v])
                    # Update rows v and w
                    for row, col in [(v, w), (w, v)]:
                        dq_heap_row = dq_heap[row]
                        # Update dict for v,w only (u is removed below)
                        elci = G.edges(list(communities[row]))
                        elcj = G.edges(list(communities[col]))
                        join=intersection(elci,elcj)
                        if len(join)==0:
                            continue   
                        eij=len(join)/numedges    
                        ai = len(G.edges(communities[row]))/numedges
                        aj = len(G.edges(communities[col]))/numedges
                        dq_vw= 2*(eij-ai*aj) 
                        #dq_dict[u][v] = dq
                        #dq_dict[v][u] = dq

                        dq_dict[row][col] = dq_vw
                        # Save old max of per-row heap
                        if len(dq_heap_row) > 0:
                            d_oldmax = dq_heap_row.heap[0]
                        else:
                            d_oldmax = None
                        # Add/update heaps
                        d = (row, col)
                        d_negdq = -dq_vw
                        # Save old value for finding heap index
                        if w in v_nbrs:
                            # Update existing element in per-row heap
                            dq_heap_row.update(d, d, priority=d_negdq)
                        else:
                            # We're creating a new nonzero element, add to heap
                            dq_heap_row.push(d, priority=d_negdq)
                        # Update heap of row maxes if necessary
                        if d_oldmax is None:
                            # No entries previously in this row, push new max
                            H.push(d, priority=d_negdq)
                        else:
                            # We've updated an entry in this row, has the max changed?
                            row_max = dq_heap_row.heap[0]
                            if d_oldmax != row_max or d_oldmax.priority != row_max.priority:
                                H.update(d_oldmax, row_max)

                # Remove row/col u from dq_dict matrix
                for w in dq_dict[u]:
                    # Remove from dict
                    dq_old = dq_dict[w][u]
                    del dq_dict[w][u]
                    # Remove from heaps if we haven't already
                    if w != v:
                        # Remove both row and column
                        for row, col in [(w, u), (u, w)]:
                            dq_heap_row = dq_heap[row]
                            # Check if replaced dq is row max
                            d_old = (row, col)
                            if dq_heap_row.heap[0] == d_old:
                                # Update per-row heap and heap of row maxes
                                dq_heap_row.remove(d_old)
                                H.remove(d_old)
                                # Update row max
                                if len(dq_heap_row) > 0:
                                    H.push(dq_heap_row.heap[0])
                            else:
                                # Only update per-row heap
                                dq_heap_row.remove(d_old)

                del dq_dict[u]
                # Mark row u as deleted, but keep placeholder
                dq_heap[u] = MappedQueue()

                #cetak_dq_dict(dq_dict)
                #cdq_heap = copy.deepcopy(dq_heap)
                #cetak_dq_heap(cdq_heap)
            else:
                berhenti=True
                #print("berhenti")

    #print(communities)
    #print("==== SELESAI ITERASI ==================")
        
    return communities, berhenti
    

In [10]:
def greedy_modularity(G, iterasi=10, modepecah=0, modelepas=0):
    global mulai
    communities = {n: frozenset([n]) for n in G}
    #print(communities)
    GG=G
    mulai=True
    #print(communities)
    n=len(G.nodes())*iterasi
    komunitasterbaik=communities;
    modularitasterbaik = hitungmodularitas(G,komunitasterbaik)
    #print(modularitasterbaik, reformat2(G,komunitasterbaik))

    #pil = [0,0,0,0,0,0,0,1,0,0,0]
    berhenti=False
    for i in range(n):
        if ((i % 100)==0):
            print(".",end="")
        #print(i, mulai, communities) 
        if i<len(G.nodes()):
            pilihan=0
        else:
            pilihan=random.choices([0,1,2], [0.9,0.05, 0.05])[0]
        #pilihan=random.choices([0,1], [0.95,0.05])[0]
        #pilihan = pil[i]

        if pilihan == 0:
            communities,  berhenti=greedy_modularity_iteration(G, communities)
            #print(i,communities)
            mulai=False
        if berhenti:
            komunitasterbaik, modularitasterbaik= updatekomunitasterbaik(G,communities,komunitasterbaik, modularitasterbaik);
            #print("update terbaik",i,modularitasterbaik)
        if (pilihan ==1 or berhenti):
            if modepecah==0: 
                communities = pecah_random(communities)
            elif modepecah==1:
                communities = pecah_weak(communities)                
            elif modepecah==2:
                communities = pecah_high_conductance(communities)                
            elif modepecah==3:
                communities = pecah_low_tpr(communities)                
            else: #modepecah==4
                communities = pecah_low_edge_density(communities)                                
            mulai=True
        if (pilihan ==2 or berhenti):
            if modelepas==0:
                communities = lepas_random(communities)
            elif modelepas==1:
                communities = lepas_weak_node(communities)
            elif modelepas==2:
                communities = lepas_non_triad_node(communities)
            else: # modelepas==3:
                communities = lepas_low_embeddedness_node(communities)
            mulai=True            
    #print(komunitasterbaik)
    return komunitasterbaik

In [11]:
G=nx.Graph()
#G.add_edges_from([(0,1),(1,2),(2,0),(2,3),(3,4),(4,5),(5,3)])

G.add_edges_from ([(0, 1), (0, 2), (0, 3), (1, 3), (1, 2), (2, 6), (2, 3), (4, 5), (4, 6), (5, 6)])

datasets=["Karate","Dolphins", "lesmis","polbooks" ,"adjnoun","football"] 
          

listmodepecah = ["pecah_random", "pecah_weak","pecah_high_conductance", "pecah_low_tpr", "pecah_low_edge_density"]
listmodelepas=["lepas_random","lepas_weak_node", "lepas_non_triad_node", "lepas_low_embeddedness_node"]
dfcom = pd.DataFrame(columns = [ 'dataset','simpul','iterasi','modepecah','modelepas','modularity','komunitas', 'n_komunitas','time'])
#dataset=datasets[0]
iterasi=20
for dataset in datasets:
    G = nx.read_gml(dataset+".gml", label = 'id')
    #pos = nx.spring_layout(G)
    #nx.draw(G, with_labels=True, node_size=500)
    for modepecah in range(5):
        for modelepas in range(4):
            try:
                awal = time.time()
                komunitasterbaik=greedy_modularity(G,iterasi,modepecah,modelepas)
                akhir= time.time()
                bestcoms= reformat2(G,komunitasterbaik)
                bestmodularitas = nx_comm.modularity(G,bestcoms)
                waktu= akhir-awal
                dftemp = {'dataset':dataset, 
                  'simpul':len(G.nodes()),
                  'iterasi':iterasi,  
                  'modepecah':listmodepecah[modepecah],
                  'modelepas':listmodelepas[modelepas],
                  'modularity':bestmodularitas,
                  'komunitas':bestcoms,
                  'n_komunitas':len(bestcoms),
                  'time':waktu}
                dfcom = dfcom.append(dftemp,ignore_index=True) 
                print(dataset, len(G.nodes()), bestmodularitas,listmodepecah[modepecah], listmodelepas[modelepas],  waktu)
            except:
                print(dataset, len(G.nodes()), "-", listmodepecah[modepecah], listmodelepas[modelepas], "-")
    #print("modularitas %.4f" % bestmodularitas)
    #print("waktu %.4f detik" % selisih )



.......Karate 34 0.41880341880341876 pecah_random lepas_random 0.5794482231140137
.......Karate 34 0.41880341880341876 pecah_random lepas_weak_node 0.6013906002044678
.......Karate 34 0.41978961209730437 pecah_random lepas_non_triad_node 0.5904490947723389
.......Karate 34 0.41559829059829057 pecah_random lepas_low_embeddedness_node 0.5993680953979492
.......Karate 34 0.41978961209730437 pecah_weak lepas_random 0.6761643886566162
.......Karate 34 0.41880341880341876 pecah_weak lepas_weak_node 0.640315055847168
.......Karate 34 0.41978961209730437 pecah_weak lepas_non_triad_node 0.6293158531188965
.......Karate 34 0.41880341880341876 pecah_weak lepas_low_embeddedness_node 0.6272933483123779
.......Karate 34 0.41978961209730437 pecah_high_conductance lepas_random 0.5116274356842041
.......Karate 34 0.388560157790927 pecah_high_conductance lepas_weak_node 0.5435488224029541
.......Karate 34 0.3921761998685076 pecah_high_conductance lepas_non_triad_node 0.5475351810455322
.......Karate 34 

.......................adjnoun 112 0.29841937716262973 pecah_random lepas_non_triad_node 11.2329421043396
.......................adjnoun 112 0.29302975778546714 pecah_random lepas_low_embeddedness_node 9.817140579223633
.......................adjnoun 112 0.3025217993079585 pecah_weak lepas_random 8.585837841033936
.......................adjnoun 112 0.29167058823529407 pecah_weak lepas_weak_node 11.214717864990234
.......................adjnoun 112 0.2955100346020761 pecah_weak lepas_non_triad_node 9.461357355117798
.......................adjnoun 112 0.2846782006920415 pecah_weak lepas_low_embeddedness_node 11.724502801895142
.......................adjnoun 112 0.2913771626297578 pecah_high_conductance lepas_random 11.018991947174072
.......................adjnoun 112 0.2903861591695502 pecah_high_conductance lepas_weak_node 12.130197286605835
.......................adjnoun 112 0.299042214532872 pecah_high_conductance lepas_non_triad_node 9.98858380317688
.......................adjnoun 1

In [12]:
dfcom

,dataset,simpul,iterasi,modepecah,modelepas,modularity,komunitas,n_komunitas,time
0,Karate,34,20,pecah_random,lepas_random,0.418803,"[[32, 24, 25, 26, 28, 29], [1, 2, 3, 4, 8, 10,...",4,0.579448
1,Karate,34,20,pecah_random,lepas_weak_node,0.418803,"[[5, 6, 7, 11, 17], [1, 2, 3, 4, 8, 10, 12, 13...",4,0.601391
2,Karate,34,20,pecah_random,lepas_non_triad_node,0.419790,"[[1, 2, 3, 4, 8, 12, 13, 14, 18, 20, 22], [32,...",4,0.590449
3,Karate,34,20,pecah_random,lepas_low_embeddedness_node,0.415598,"[[32, 25, 26, 29], [1, 2, 3, 4, 8, 10, 12, 13,...",4,0.599368
4,Karate,34,20,pecah_weak,lepas_random,0.419790,"[[5, 6, 7, 11, 17], [1, 2, 3, 4, 8, 12, 13, 14...",4,0.676164
...,...,...,...,...,...,...,...,...,...
115,football,115,20,pecah_low_tpr,lepas_low_embeddedness_node,0.603162,"[[34, 99, 36, 38, 71, 42, 43, 12, 14, 18, 85, ...",8,16.620881
116,football,115,20,pecah_low_edge_density,lepas_random,0.573851,"[[73, 110, 46, 49, 114, 83, 53, 88], [64, 32, ...",8,18.954829
117,football,115,20,pecah_low_edge_density,lepas_weak_node,0.573126,"[[34, 99, 38, 71, 43, 12, 14, 18, 85, 54, 26, ...",6,22.070292
118,football,115,20,pecah_low_edge_density,lepas_non_triad_node,0.599335,"[[64, 32, 2, 100, 6, 39, 106, 13, 47, 15, 60],...",7,20.033635


In [13]:
dfcom.to_excel("hasil Databases.xlsx")